# Twitter Sentiment Analysis  

## Here we shall use the Sentiment 140 Dataset. More about this dataset has been provided in the ReadMe File.


## Importing Dependencies

In [2]:
import numpy as np
import pandas as pd
import re
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score,classification_report

In [3]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /usr/share/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

### Stopwords are those words that do not have a particular meaning or doesnt tell much of the sentence or tweet is positive, neutral or negative. So here we download the list of stopwords and try and then later delete them from the tweet.

In [4]:
#printing the stopwords in English

print(stopwords.words('english'))

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', '

## Data processing

In [5]:
# loading the data:
twitter_data = pd.read_csv('/kaggle/input/sentiment140/training.1600000.processed.noemoticon.csv',encoding = 'ISO-8859-1')

#Latin-1, also called ISO-8859-1, is an 8-bit character set endorsed by the International Organization for Standardization (ISO) and represents the alphabets of Western European languages

In [6]:
# Checking the number of rows and columns
twitter_data.shape

(1599999, 6)

In [7]:
twitter_data.head()

,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, that's a bummer. You shoulda got David Carr of Third Day to do it. ;D"
0,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...
1,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...
2,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire
3,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all...."
4,0,1467811372,Mon Apr 06 22:20:00 PDT 2009,NO_QUERY,joy_wolf,@Kwesidei not the whole crew


In [8]:
# naming the table headers

headers = ['target','id','data','flag','user','text']
twitter_data = pd.read_csv('/kaggle/input/sentiment140/training.1600000.processed.noemoticon.csv',names = headers, encoding = 'ISO-8859-1')

In [9]:
twitter_data.head()

,target,id,data,flag,user,text
0,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, t..."
1,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...
2,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...
3,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire
4,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all...."


In [10]:
# Checking if there are any null or missing values

twitter_data.isnull().sum()


target    0
id        0
data      0
flag      0
user      0
text      0
dtype: int64

In [11]:
# checking the distribution of target column
twitter_data['target'].value_counts()

target
0    800000
4    800000
Name: count, dtype: int64

In [12]:
twitter_data.replace({'target':{4:1}},inplace = True)

In [13]:
twitter_data['target'].value_counts()

## 0 -> Negative tweet 
## 1 -> Positive Tweet

target
0    800000
1    800000
Name: count, dtype: int64

## Stemming

### Since the dataset is very huge with 1.6 million tweets, so as to make it less complex we use stemming. Stemming is the process of reducing a words to its Root word

### example: actor,actress,acting = act ; dancer,dancing = dance and so on. 

In [14]:
port_stem = PorterStemmer()

In [15]:
from tqdm import tqdm

In [16]:
def stemming(content):
    stemmed_content = re.sub('[^a-zA-Z]',' ',content) #Removing all the things that are not alphabets eg: @,numbers etc.
    stemmed_counter = stemmed_content.lower() #converts capital letters to small
    stemmed_content = stemmed_content.split() #split the words
    stemmed_content = [port_stem.stem(word) for word in stemmed_content if not word in stopwords.words('english')]#reduce word to root word and making sure the word is not a stopword.
    stemmed_content = ' '.join(stemmed_content) #joining the words to form the tweet again.
    
    return stemmed_content

In [17]:
twitter_data['stemmed_content'] = [stemming(text) for text in tqdm(twitter_data['text'])]

100%|██████████| 1600000/1600000 [51:55<00:00, 513.63it/s]


In [18]:
twitter_data.head()

,target,id,data,flag,user,text,stemmed_content
0,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, t...",switchfoot http twitpic com zl awww bummer you...
1,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...,upset updat facebook text might cri result sch...
2,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...,kenichan I dive mani time ball manag save the ...
3,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire,whole bodi feel itchi like fire
4,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all....",nationwideclass behav mad I see


In [19]:
print(twitter_data['stemmed_content'])

0          switchfoot http twitpic com zl awww bummer you...
1          upset updat facebook text might cri result sch...
2          kenichan I dive mani time ball manag save the ...
3                            whole bodi feel itchi like fire
4                            nationwideclass behav mad I see
                                 ...                        
1599995                 just woke have school best feel ever
1599996    thewdb com veri cool hear old walt interview h...
1599997                     are readi mojo makeov ask detail
1599998    happi th birthday boo alll time tupac amaru sh...
1599999    happi charitytuesday thenspcc sparkschar speak...
Name: stemmed_content, Length: 1600000, dtype: object


In [20]:
print(twitter_data['target'])

0          0
1          0
2          0
3          0
4          0
          ..
1599995    1
1599996    1
1599997    1
1599998    1
1599999    1
Name: target, Length: 1600000, dtype: int64


In [21]:
# Seperating data and label and preparing it for training.

X = twitter_data['stemmed_content'].values
y = twitter_data['target'].values
print(X)
print(y)

['switchfoot http twitpic com zl awww bummer you shoulda got david carr third day D'
 'upset updat facebook text might cri result school today also blah'
 'kenichan I dive mani time ball manag save the rest go bound' ...
 'are readi mojo makeov ask detail'
 'happi th birthday boo alll time tupac amaru shakur'
 'happi charitytuesday thenspcc sparkschar speakinguph H']
[0 0 0 ... 1 1 1]


# Splitting the data to training and test data

In [22]:
X_train, X_test,y_train,y_test = train_test_split(X,y,test_size = 0.2,stratify = y,random_state = 2)
print(X.shape,X_train.shape,X_test.shape)
print(y.shape,y_train.shape,y_test.shape)

(1600000,) (1280000,) (320000,)
(1600000,) (1280000,) (320000,)


### The importance of the word is checked and a numerical value is given to it. For example lets say in the collection of tweets the word happy is mostly used in positive tweets and is frequently used. So accordingly denoting the importance of the word a numerical value is assigned to it.

In [23]:
print(X_train)

['watch saw iv drink lil wine' 'hatermagazin I'
 'even though favourit drink think vodka coke wipe mind time think im gonna find new drink'
 ... 'eager monday afternoon'
 'hope everyon mother great day can wait hear guy store tomorrow'
 'I love wake folger too bad voic deeper']


In [24]:
print(X_test)

['mmangen M fine I much time chat twitter hubbi back summer amp tend domin free time'
 'ah may show w ruth kim amp geoffrey sanhueza'
 'ishatara mayb bay area thang dammit' ...
 'destini nevertheless hooray member wonder safe trip' 'not feel well'
 'supersandro thank']


In [25]:
# converting the textual data to numerical data

vectorizer = TfidfVectorizer() 
X_train = vectorizer.fit_transform(X_train)
X_test = vectorizer.transform(X_test)


In [26]:
print(X_train)

  (0, 443091)	0.4484755317023172
  (0, 235060)	0.41996827700291095
  (0, 109311)	0.3753708587402299
  (0, 185208)	0.5277679060576009
  (0, 354563)	0.3588091611460021
  (0, 436736)	0.2725987626483838
  (1, 160645)	1.0
  (2, 288488)	0.1673801447945276
  (2, 132317)	0.20291428205045578
  (2, 150722)	0.18805437673647912
  (2, 178072)	0.16191467578116914
  (2, 409165)	0.15170562659204556
  (2, 266745)	0.2412526672795364
  (2, 443455)	0.33488823001589063
  (2, 77931)	0.3128672120237835
  (2, 433582)	0.32968741386905526
  (2, 406421)	0.32108169269258585
  (2, 129417)	0.29076646660098543
  (2, 407323)	0.1871091779838034
  (2, 124490)	0.18923156633380322
  (2, 109311)	0.45915639201149566
  (3, 172432)	0.37464146922154384
  (3, 411550)	0.27089772444087873
  (3, 388648)	0.3940776331458846
  (3, 56478)	0.5200465453608686
  :	:
  (1279996, 318323)	0.21254698865277746
  (1279996, 237914)	0.22365675600992338
  (1279996, 291096)	0.17981734369155505
  (1279996, 412575)	0.18967045002348676
  (1279997, 1

In [27]:
print(X_test)

  (0, 421006)	0.17915624523539803
  (0, 409165)	0.31430470598079707
  (0, 398928)	0.3491043873264266
  (0, 388370)	0.21985076072061732
  (0, 279098)	0.1782518010910344
  (0, 271032)	0.4535662391658828
  (0, 171389)	0.2805816206356073
  (0, 138171)	0.23688292264071403
  (0, 132370)	0.25525488955578596
  (0, 106074)	0.3655545001090455
  (0, 67830)	0.2680037527082731
  (0, 31169)	0.16247724180521766
  (0, 15110)	0.1719352837797837
  (1, 366225)	0.24595562404108307
  (1, 348155)	0.4739279595416274
  (1, 256792)	0.28751585696559306
  (1, 217577)	0.40288153995289894
  (1, 145400)	0.575262969264869
  (1, 15110)	0.211037449588008
  (1, 6463)	0.30733520460524466
  (2, 400643)	0.4317732461913093
  (2, 256849)	0.2564939661498775
  (2, 183325)	0.5892069252021465
  (2, 89451)	0.36340369428387626
  (2, 34402)	0.37916255084357414
  :	:
  (319995, 406193)	0.2608397980426395
  (319995, 324516)	0.33450801025731414
  (319995, 315833)	0.2636898530520663
  (319995, 296681)	0.369625344051842
  (319995, 2585

## Training the machine learning model

## Using logistic regression

In [28]:
model = LogisticRegression(max_iter = 1000)

In [29]:
model.fit(X_train,y_train)

LogisticRegression(max_iter=1000)

## Model Evaluation

In [30]:
# accuracy score on the training data
X_train_pred = model.predict(X_train)
training_data_accuracy = accuracy_score(y_train,X_train_pred)
print('Accuracy on the training data:',training_data_accuracy)

Accuracy on the training data: 0.8120890625


In [37]:
X_test_prediction = model.predict(X_test)
test_data_accuracy = accuracy_score(y_test,X_test_prediction)
print('Accuracy on the test data:',test_data_accuracy)

Accuracy on the test data: 0.78156875


### Accuracy on training data: 0.81%
### Accuracy on the test data: 0.78%

# Saving the model

In [32]:
import pickle

In [33]:
filename = 'trained_model.sav'
pickle.dump(model,open(filename,'wb')) #Writing in the form of binary format

## Using saved model for future predictions

In [34]:
#loading the model
load_model = pickle.load(open('/kaggle/working/trained_model.sav','rb'))

In [35]:
X_new = X_test[100]
print(X_new)

  (0, 389982)	0.22047344888749912
  (0, 387448)	0.17194330665682203
  (0, 350168)	0.18694002963635611
  (0, 341484)	0.23373589790439547
  (0, 339224)	0.3123226332985506
  (0, 286676)	0.3263342189828755
  (0, 276152)	0.2128030069260303
  (0, 182506)	0.2740391209232345
  (0, 166939)	0.23706563188730737
  (0, 145995)	0.11302339110709633
  (0, 127250)	0.2985162910610305
  (0, 95779)	0.21700170510343478
  (0, 49854)	0.29507158396791555
  (0, 33911)	0.36737724639674835
  (0, 23295)	0.2839304598622885


In [36]:
print(y_test[100])

prediction = model.predict(X_new)
print(prediction)

if(prediction[0] == 0):
    print('Negative Tweet')
else:
    print('Positive Tweet')

0
[0]
Negative Tweet
